In [1]:
from time import time
from math import ceil, floor, sqrt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_percentage_error, mean_absolute_error

from sklearn.preprocessing import MinMaxScaler
from scipy import stats
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 

In [7]:
# set global variables for training
seed = 7
train_test_split = 0.8
pd.set_option('display.max_columns', None)

In [ ]:
import math
def p4k_exists(row):
    if math.isnan(row['score_p4k']):
        return 0
    else:
        return 1
df_train['reviewed_p4k'] = df_train.apply(p4k_exists, axis=1)
df_train

In [3]:
# load datasets

df_500greatest = pd.read_csv('datasets/rolling_stone_top_500_albums_2020.csv')
df_discogs = pd.read_csv('datasets/discogs_most_collected.csv')
df_discogs_most_wanted = pd.read_csv('datasets/all_release_data_most_wanted_9799.csv')
df_p4k_spotify = pd.read_csv('datasets/p4kXspotify.csv', encoding = "unicode_escape")
df_discogs_p4k_spotify = pd.read_csv('datasets/all_release_data_p4k_spotify_filled_partial.csv', encoding = "unicode_escape")

print("datasets loaded")

datasets loaded


In [8]:
df_discogs_p4k_spotify

,Unnamed: 0,most_collected_rank,artist_x,title_x,url,year,date,genres,styles,countries,formats,issue_type,limited,picture_disc,box_set,numbered,test_pressing,promo,channels,gatefold,colored,haves,wants,avg_rating,num_ratings,price_median,price_lowest,price_highest,influential_album,title_lower,artist_lower,artist_y,title_y,reviewauthor,score,releaseyear,reviewdate,recordlabel,genre,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0,0,Daft Punk,Random Access Memories,https://www.discogs.com/release/4570366-Daft-P...,2013,2013-05-17,"['Electronic', 'Funk / Soul', 'Pop']","['Disco', 'Funk', 'Synth-pop']","UK, Europe & US","[['2 x vinyl', 'lp', 'album', 'stereo', '180 g...",1,0,0,0,0,0,0,2,0,0,53521,18838,4.56,6337,45.00,21.11,189.99,0,random access memories,daft punk,Daft Punk,Random Access Memories,Mark Richardson,8.8,2013.0,May 20 2013,Columbia,Electronic,0.686462,0.585846,7.538462,-10.704077,0.044315,0.287906,0.373620,0.132877,0.488231,114.471077
1,1,1,Kendrick Lamar,"Good Kid, M.A.A.d City",https://www.discogs.com/release/3975953-Kendri...,2012,2012-10-22,['Hip Hop'],['Conscious'],US,"[['2 x vinyl', 'lp', 'album', 'deluxe edition']]",1,0,0,0,0,0,0,1,0,0,41176,12493,4.66,2957,20.90,13.29,35.76,0,"good kid, m.a.a.d city",kendrick lamar,Kendrick Lamar,"good kid, m.A.A.d city",Jayson Greene,9.5,2012.0,October 23 2012,Interscope,Rap,0.601294,0.676706,4.529412,-7.811941,0.268318,0.123219,0.000669,0.311412,0.398206,116.045059
2,2,2,Michael Jackson,Thriller,https://www.discogs.com/release/2911293-Michae...,1982,1982-11-30,"['Funk / Soul', 'Pop']","['Contemporary R&B', 'Disco', 'Soul']",US,"[['vinyl', 'lp', 'album', 'stereo', 'pitman pr...",1,0,0,0,0,0,0,2,1,1,40452,9254,4.48,2744,15.06,4.53,99.99,1,thriller,michael jackson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.727100,0.585233,6.266667,-9.707967,0.349890,0.375693,0.044753,0.296793,0.608980,110.307933
3,3,3,Pink Floyd,The Dark Side Of The Moon,https://www.discogs.com/release/9287809-Pink-F...,2016,2016-11-04,['Rock'],"['Prog Rock', 'Psychedelic Rock', 'Classic Rock']",Europe,"[['vinyl', 'lp', 'album', 'reissue', 'remaster...",2,0,0,0,0,0,0,2,1,0,38314,3961,4.83,4201,23.62,19.90,33.25,0,the dark side of the moon,pink floyd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.383000,0.400450,6.000000,-15.701900,0.058660,0.390310,0.595929,0.237680,0.272830,120.066200
4,4,4,Fleetwood Mac,Rumours,https://www.discogs.com/release/526351-Fleetwo...,1977,1977,['Rock'],"['Soft Rock', 'Classic Rock']",US,"[['vinyl', 'lp', 'album', 'los angeles pressin...",1,0,0,0,0,0,0,1,0,1,35108,8351,4.50,2713,21.84,2.85,189.05,1,rumours,fleetwood mac,Fleetwood Mac,Rumours,Jessica Hopper,10.0,1977.0,February 8 2013,Rhino,Rock,0.857000,0.812000,10.000000,-7.362000,0.055500,0.084400,0.000000,0.063800,0.684000,106.446000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8923,8923,9998,Minor Threat,Minor Threat,https://www.discogs.com/release/1267538-Minor-...,1987,1987,['Rock'],"['Hardcore', 'Punk']",US,"[['vinyl', 'lp', 'compilation', 'repress', '$5...",2,0,0,0,0,0,0,1,0,1,2290,532,4.59,160,34.99,13.00,47.50,0,minor threat,minor threat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.294071,0.941286,3.642857,-6.782429,0.097100,0.028782,0.122042,0.312636,0.632857,148.792857
8924,8924,9999,Carole King,Wrap Around Joy,https://www.discogs.com/release/1602932-Carole...,1974,1974,['Rock'],['Pop Rock'],US,"[['vinyl', 'lp', 'album', 'terre haute pressin...",1,0,0,0,0,0,0,1,0,1,2289,82,3.58,111,2.57,0.99,8.00,0,wrap around joy,carole king,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.547833,0.452833,3.500000,-13.260250,0.029158,0.099133,0.000132,0.129717,0.650167,119.629833
8925,8925,10000,Tom Jones,This Is Tom Jones,https://www.discogs.com/release/1804019-Tom-Jo...,1969,1969,['Pop'],"['Ballad', 'Vocal']",US,"[['vinyl', 'lp', 'album']]",1,0,0,0,0,0,0,1,0,0,2292,31,3.50,84,2

In [10]:
df_train = df_discogs_p4k_spotify[["artist_x","title_x","year","genres","styles","issue_type","limited","picture_disc","numbered","promo","channels","gatefold","colored","haves","wants","avg_rating","num_ratings","price_median","influential_album","score","danceability","energy","key","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo"]]

In [12]:
df_train.rename(columns={'artist_x': 'artist', 'title_x': 'title',"score":"score_p4k"}, inplace=True)

/Users/bubu/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [15]:
df_train["influential_album"].sum()

1076

# Treatments
## 1. Year variable 
1. Some releases have different format for the date in yyyy-mm format (Example : https://www.discogs.com/release/10241165-Fiona-Apple-Tidal )
2. Some releases have the year of release not available on the metadata section, but in `notes` section. (Example : https://www.discogs.com/release/7345404-Kendrick-Lamar-Good-Kid-mAAd-City and https://www.discogs.com/release/403972-Red-Hot-Chili-Peppers-Californication) typically the year is found in Notes near ℗ & © symbols
3. Some releases don't have the release available on the release page. For releases with only one version, other releases are used to determine the year of release (Example : https://www.discogs.com/release/11743768-Kanye-West-The-College-Dropout )
4. Some releases don't have the information at all, and there's a user discussion on the release entry on finding the exact date of release. Since such issues are likely to be re-issues, the latest of year of the available other releases is taken

## 2. Artist
1. Some artist names contain (2) due to discogs database guidelines, this is removed
2. Some artist names contain the problematic $$ at the end of their names that's problematic for python, it's replaced with SS instead.

## 3. Formats
5 releases contained no details on the format, therefore removed

## 4. Market statistics
5 releases contained no market statistics, therefore removed

## 4. Price
EUR symbol removed


In [16]:
# create influential album column
influentials = 0

def findAlbuminRSDataset(album):
    global influentials
    if df_500greatest.loc[df_500greatest['Album'] == album].empty:
        return 0
    else:
        influentials += 1
        return 1

df_train['influential_album2020'] = df_train['title'].apply(findAlbuminRSDataset)
df_train.influential_album.astype("category")
print("Created influential album variable from Rolling Stones 500 Greatest Album list. Found ", str(influentials), " releases.")

Created influential album variable from Rolling Stones 500 Greatest Album list. Found  1071  releases.


/var/folders/2k/lqwk1p1108jctdxn9x58f5qm0000gn/T/ipykernel_31613/910923885.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['influential_album2020'] = df_train['title'].apply(findAlbuminRSDataset)


In [17]:
# create album review text column
influentials = 0

def findAlbumTextinRSDataset(album):
    global influentials
    if df_500greatest.loc[df_500greatest['Album'] == album].empty:
        return "empty"
    else:
        influentials += 1
        return df_500greatest.loc[df_500greatest['Album'] == album].iloc[0]['Description']

df_train['influential_album_text'] = df_train['title'].apply(findAlbumTextinRSDataset)
print("Created influential album variable from Rolling Stones 500 Greatest Album list. Added reviews for ", str(influentials), " releases.")

Created influential album variable from Rolling Stones 500 Greatest Album list. Added reviews for  1071  releases.


/var/folders/2k/lqwk1p1108jctdxn9x58f5qm0000gn/T/ipykernel_31613/3565334579.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['influential_album_text'] = df_train['title'].apply(findAlbumTextinRSDataset)


In [20]:
df_train.isnull().sum()

artist                       0
title                        0
year                         0
genres                       0
styles                       0
issue_type                   0
limited                      0
picture_disc                 0
numbered                     0
promo                        0
channels                     0
gatefold                     0
colored                      0
haves                        0
wants                        0
avg_rating                   0
num_ratings                  0
price_median                 0
influential_album            0
score_p4k                 6990
danceability               794
energy                     794
key                        794
loudness                   794
speechiness                794
acousticness               794
instrumentalness           794
liveness                   794
valence                    794
tempo                      794
influential_album2020        0
influential_album_text       0
dtype: i

In [21]:
# convert all numerical columns to numeric type
df_train["year"] = pd.to_numeric(df_train["year"], errors='coerce')
df_train["price_median"] = pd.to_numeric(df_train["price_median"], errors='coerce')
df_train["haves"] = pd.to_numeric(df_train["haves"], errors='coerce')
df_train["wants"] = pd.to_numeric(df_train["wants"], errors='coerce')
df_train["avg_rating"] = pd.to_numeric(df_train["avg_rating"], errors='coerce')
df_train["num_ratings"] = pd.to_numeric(df_train["num_ratings"], errors='coerce')

/var/folders/2k/lqwk1p1108jctdxn9x58f5qm0000gn/T/ipykernel_31613/3095961432.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["year"] = pd.to_numeric(df_train["year"], errors='coerce')
/var/folders/2k/lqwk1p1108jctdxn9x58f5qm0000gn/T/ipykernel_31613/3095961432.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["price_median"] = pd.to_numeric(df_train["price_median"], errors='coerce')
/var/folders/2k/lqwk1p1108jctdxn9x58f5qm0000gn/T/ipykernel_31613/3095961432.py:4: SettingWithCopyWarning

In [23]:
df_train.dtypes

artist                     object
title                      object
year                        int64
genres                     object
styles                     object
issue_type                  int64
limited                     int64
picture_disc                int64
numbered                    int64
promo                       int64
channels                    int64
gatefold                    int64
colored                     int64
haves                       int64
wants                       int64
avg_rating                float64
num_ratings                 int64
price_median              float64
influential_album           int64
score_p4k                 float64
danceability              float64
energy                    float64
key                       float64
loudness                  float64
speechiness               float64
acousticness              float64
instrumentalness          float64
liveness                  float64
valence                   float64
tempo         

In [ ]:
# convert all numerical columns to numeric type
df_discogs["year"] = pd.to_numeric(df_discogs["year"], errors='coerce')
df_discogs["price_median"] = pd.to_numeric(df_discogs["price_median"], errors='coerce')
df_discogs["price_highest"] = pd.to_numeric(df_discogs["price_highest"], errors='coerce')
df_discogs["price_lowest"] = pd.to_numeric(df_discogs["price_lowest"], errors='coerce')
df_discogs["haves"] = pd.to_numeric(df_discogs["haves"], errors='coerce')
df_discogs["wants"] = pd.to_numeric(df_discogs["wants"], errors='coerce')
df_discogs["avg_rating"] = pd.to_numeric(df_discogs["avg_rating"], errors='coerce')
df_discogs["num_ratings"] = pd.to_numeric(df_discogs["num_ratings"], errors='coerce')

In [ ]:
df_discogs_p4k_spotify["year"] = pd.to_numeric(df_discogs["year"], errors='coerce')
df_discogs_p4k_spotify["price_median"] = pd.to_numeric(df_discogs["price_median"], errors='coerce')
df_discogs_p4k_spotify["price_highest"] = pd.to_numeric(df_discogs["price_highest"], errors='coerce')
df_discogs_p4k_spotify["price_lowest"] = pd.to_numeric(df_discogs["price_lowest"], errors='coerce')
df_discogs_p4k_spotify["haves"] = pd.to_numeric(df_discogs["haves"], errors='coerce')
df_discogs_p4k_spotify["wants"] = pd.to_numeric(df_discogs["wants"], errors='coerce')
df_discogs_p4k_spotify["avg_rating"] = pd.to_numeric(df_discogs["avg_rating"], errors='coerce')
df_discogs_p4k_spotify["num_ratings"] = pd.to_numeric(df_discogs["num_ratings"], errors='coerce')

In [ ]:
# convert year to age
df_discogs["age"] = (2022 - df_discogs["year"]) + 1
df_discogs

In [24]:
# convert year to age
df_train["age"] = (2022 - df_train["year"]) + 1
df_train

/var/folders/2k/lqwk1p1108jctdxn9x58f5qm0000gn/T/ipykernel_31613/3060249920.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["age"] = (2022 - df_train["year"]) + 1


,artist,title,year,genres,styles,issue_type,limited,picture_disc,numbered,promo,channels,gatefold,colored,haves,wants,avg_rating,num_ratings,price_median,influential_album,score_p4k,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,influential_album2020,influential_album_text,age
0,Daft Punk,Random Access Memories,2013,"['Electronic', 'Funk / Soul', 'Pop']","['Disco', 'Funk', 'Synth-pop']",1,0,0,0,0,2,0,0,53521,18838,4.56,6337,45.00,0,8.8,0.686462,0.585846,7.538462,-10.704077,0.044315,0.287906,0.373620,0.132877,0.488231,114.471077,1,Having played a massive role in the rise of ED...,10
1,Kendrick Lamar,"Good Kid, M.A.A.d City",2012,['Hip Hop'],['Conscious'],1,0,0,0,0,1,0,0,41176,12493,4.66,2957,20.90,0,9.5,0.601294,0.676706,4.529412,-7.811941,0.268318,0.123219,0.000669,0.311412,0.398206,116.045059,0,empty,11
2,Michael Jackson,Thriller,1982,"['Funk / Soul', 'Pop']","['Contemporary R&B', 'Disco', 'Soul']",1,0,0,0,0,2,1,1,40452,9254,4.48,2744,15.06,1,NaN,0.727100,0.585233,6.266667,-9.707967,0.349890,0.375693,0.044753,0.296793,0.608980,110.307933,1,Michael Jackson towered over the Eighties the ...,41
3,Pink Floyd,The Dark Side Of The Moon,2016,['Rock'],"['Prog Rock', 'Psychedelic Rock', 'Classic Rock']",2,0,0,0,0,2,1,0,38314,3961,4.83,4201,23.62,0,NaN,0.383000,0.400450,6.000000,-15.701900,0.058660,0.390310,0.595929,0.237680,0.272830,120.066200,0,empty,7
4,Fleetwood Mac,Rumours,1977,['Rock'],"['Soft Rock', 'Classic Rock']",1,0,0,0,0,1,0,1,35108,8351,4.50,2713,21.84,1,10.0,0.857000,0.812000,10.000000,-7.362000,0.055500,0.084400,0.000000,0.063800,0.684000,106.446000,1,"With Rumours, Fleetwood Mac turned private tur...",46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8923,Minor Threat,Minor Threat,1987,['Rock'],"['Hardcore', 'Punk']",2,0,0,0,0,1,0,1,2290,532,4.59,160,34.99,0,NaN,0.294071,0.941286,3.642857,-6.782429,0.097100,0.028782,0.122042,0.312636,0.632857,148.792857,0,empty,36
8924,Carole King,Wrap Around Joy,1974,['Rock'],['Pop Rock'],1,0,0,0,0,1,0,1,2289,82,3.58,111,2.57,0,NaN,0.547833,0.452833,3.500000,-13.260250,0.029158,0.099133,0.000132,0.129717,0.650167,119.629833,0,empty,49
8925,Tom Jones,This Is Tom Jones,1969,['Pop'],"['Ballad', 'Vocal']",1,0,0,0,0,1,0,0,2292,31,3.50,84,2.33,0,NaN,0.354667,0.553917,4.083333,-10.294583,0.047683,0.457000,0.000111,0.212875,0.524667,133.280333,0,empty,54
8926,Pixies,Bossanova,1990,['Rock'],['Indie Rock'],1,0,0,0,0,1,0,0,2291,1286,4.26,185,34.50,0,NaN,0.472857,0.629857,5.000000,-12.273357,0.036579,0.045239,0.264102,0.143621,0.666429,132.973286,0,empty,33


In [ ]:
# convert year to age
df_discogs_p4k_spotify["age"] = (2022 - df_discogs_p4k_spotify["year"]) + 1
df_discogs_p4k_spotify

In [ ]:
# fix issue type and channels vars to be one-hot encoded
df_discogs["issue_type"] = df_discogs["issue_type"] - 1
df_discogs["channels"] = df_discogs["channels"] - 1
df_discogs

In [25]:
# fix issue type and channels vars to be one-hot encoded
df_train["issue_type"] = df_train["issue_type"] - 1
df_train["channels"] = df_train["channels"] - 1
df_train

/var/folders/2k/lqwk1p1108jctdxn9x58f5qm0000gn/T/ipykernel_31613/1813185656.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["issue_type"] = df_train["issue_type"] - 1
/var/folders/2k/lqwk1p1108jctdxn9x58f5qm0000gn/T/ipykernel_31613/1813185656.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["channels"] = df_train["channels"] - 1


,artist,title,year,genres,styles,issue_type,limited,picture_disc,numbered,promo,channels,gatefold,colored,haves,wants,avg_rating,num_ratings,price_median,influential_album,score_p4k,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,influential_album2020,influential_album_text,age
0,Daft Punk,Random Access Memories,2013,"['Electronic', 'Funk / Soul', 'Pop']","['Disco', 'Funk', 'Synth-pop']",0,0,0,0,0,1,0,0,53521,18838,4.56,6337,45.00,0,8.8,0.686462,0.585846,7.538462,-10.704077,0.044315,0.287906,0.373620,0.132877,0.488231,114.471077,1,Having played a massive role in the rise of ED...,10
1,Kendrick Lamar,"Good Kid, M.A.A.d City",2012,['Hip Hop'],['Conscious'],0,0,0,0,0,0,0,0,41176,12493,4.66,2957,20.90,0,9.5,0.601294,0.676706,4.529412,-7.811941,0.268318,0.123219,0.000669,0.311412,0.398206,116.045059,0,empty,11
2,Michael Jackson,Thriller,1982,"['Funk / Soul', 'Pop']","['Contemporary R&B', 'Disco', 'Soul']",0,0,0,0,0,1,1,1,40452,9254,4.48,2744,15.06,1,NaN,0.727100,0.585233,6.266667,-9.707967,0.349890,0.375693,0.044753,0.296793,0.608980,110.307933,1,Michael Jackson towered over the Eighties the ...,41
3,Pink Floyd,The Dark Side Of The Moon,2016,['Rock'],"['Prog Rock', 'Psychedelic Rock', 'Classic Rock']",1,0,0,0,0,1,1,0,38314,3961,4.83,4201,23.62,0,NaN,0.383000,0.400450,6.000000,-15.701900,0.058660,0.390310,0.595929,0.237680,0.272830,120.066200,0,empty,7
4,Fleetwood Mac,Rumours,1977,['Rock'],"['Soft Rock', 'Classic Rock']",0,0,0,0,0,0,0,1,35108,8351,4.50,2713,21.84,1,10.0,0.857000,0.812000,10.000000,-7.362000,0.055500,0.084400,0.000000,0.063800,0.684000,106.446000,1,"With Rumours, Fleetwood Mac turned private tur...",46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8923,Minor Threat,Minor Threat,1987,['Rock'],"['Hardcore', 'Punk']",1,0,0,0,0,0,0,1,2290,532,4.59,160,34.99,0,NaN,0.294071,0.941286,3.642857,-6.782429,0.097100,0.028782,0.122042,0.312636,0.632857,148.792857,0,empty,36
8924,Carole King,Wrap Around Joy,1974,['Rock'],['Pop Rock'],0,0,0,0,0,0,0,1,2289,82,3.58,111,2.57,0,NaN,0.547833,0.452833,3.500000,-13.260250,0.029158,0.099133,0.000132,0.129717,0.650167,119.629833,0,empty,49
8925,Tom Jones,This Is Tom Jones,1969,['Pop'],"['Ballad', 'Vocal']",0,0,0,0,0,0,0,0,2292,31,3.50,84,2.33,0,NaN,0.354667,0.553917,4.083333,-10.294583,0.047683,0.457000,0.000111,0.212875,0.524667,133.280333,0,empty,54
8926,Pixies,Bossanova,1990,['Rock'],['Indie Rock'],0,0,0,0,0,0,0,0,2291,1286,4.26,185,34.50,0,NaN,0.472857,0.629857,5.000000,-12.273357,0.036579,0.045239,0.264102,0.143621,0.666429,132.973286,0,empty,33


In [ ]:
# fix issue type and channels vars to be one-hot encoded
df_discogs_p4k_spotify["issue_type"] = df_discogs_p4k_spotify["issue_type"] - 1
df_discogs_p4k_spotify["channels"] = df_discogs_p4k_spotify["channels"] - 1
df_discogs_p4k_spotify

In [ ]:
# transform targets using log-transform for better distribution
df_discogs["log_age"] = np.log(df_discogs["age"])
df_discogs["log_price_median"] = np.log(df_discogs["price_median"])
df_discogs["log_price_highest"] = np.log(df_discogs["price_highest"])
df_discogs["log_price_lowest"] = np.log(df_discogs["price_lowest"])
df_discogs["log_haves"] = np.log(df_discogs["haves"])
df_discogs["log_wants"] = np.log(df_discogs["wants"])
df_discogs["log_avg_rating"] = np.log(df_discogs["avg_rating"])
df_discogs["log_num_ratings"] = np.log(df_discogs["avg_rating"])

In [ ]:
# clean artist data
#ValueError: 
#Joey Bada$$
#Expected end of text, found '$'  (at char 9), (line:1, col:10)

df_discogs['artist'] = df_discogs['artist'].str.replace(r"\$\$","SS", regex = True)
df_discogs['artist'] = df_discogs['artist'].str.replace(r"\([0-9]\)","", regex = True)

In [26]:
# clean artist data
#ValueError: 
#Joey Bada$$
#Expected end of text, found '$'  (at char 9), (line:1, col:10)

df_train['artist'] = df_train['artist'].str.replace(r"\$\$","SS", regex = True)
df_train['title'] = df_train['title'].str.replace(r"\$\$","SS", regex = True)
df_train['artist'] = df_train['artist'].str.replace(r"\([0-9]\)","", regex = True)
df_train

/var/folders/2k/lqwk1p1108jctdxn9x58f5qm0000gn/T/ipykernel_31613/1932535892.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['artist'] = df_train['artist'].str.replace(r"\$\$","SS", regex = True)
/var/folders/2k/lqwk1p1108jctdxn9x58f5qm0000gn/T/ipykernel_31613/1932535892.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['title'] = df_train['title'].str.replace(r"\$\$","SS", regex = True)
/var/folders/2k/lqwk1p1108jctdxn9x58f5qm0000gn/T/ipykernel_31613/1932535892.py:8: SettingWithCopyW

,artist,title,year,genres,styles,issue_type,limited,picture_disc,numbered,promo,channels,gatefold,colored,haves,wants,avg_rating,num_ratings,price_median,influential_album,score_p4k,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,influential_album2020,influential_album_text,age
0,Daft Punk,Random Access Memories,2013,"['Electronic', 'Funk / Soul', 'Pop']","['Disco', 'Funk', 'Synth-pop']",0,0,0,0,0,1,0,0,53521,18838,4.56,6337,45.00,0,8.8,0.686462,0.585846,7.538462,-10.704077,0.044315,0.287906,0.373620,0.132877,0.488231,114.471077,1,Having played a massive role in the rise of ED...,10
1,Kendrick Lamar,"Good Kid, M.A.A.d City",2012,['Hip Hop'],['Conscious'],0,0,0,0,0,0,0,0,41176,12493,4.66,2957,20.90,0,9.5,0.601294,0.676706,4.529412,-7.811941,0.268318,0.123219,0.000669,0.311412,0.398206,116.045059,0,empty,11
2,Michael Jackson,Thriller,1982,"['Funk / Soul', 'Pop']","['Contemporary R&B', 'Disco', 'Soul']",0,0,0,0,0,1,1,1,40452,9254,4.48,2744,15.06,1,NaN,0.727100,0.585233,6.266667,-9.707967,0.349890,0.375693,0.044753,0.296793,0.608980,110.307933,1,Michael Jackson towered over the Eighties the ...,41
3,Pink Floyd,The Dark Side Of The Moon,2016,['Rock'],"['Prog Rock', 'Psychedelic Rock', 'Classic Rock']",1,0,0,0,0,1,1,0,38314,3961,4.83,4201,23.62,0,NaN,0.383000,0.400450,6.000000,-15.701900,0.058660,0.390310,0.595929,0.237680,0.272830,120.066200,0,empty,7
4,Fleetwood Mac,Rumours,1977,['Rock'],"['Soft Rock', 'Classic Rock']",0,0,0,0,0,0,0,1,35108,8351,4.50,2713,21.84,1,10.0,0.857000,0.812000,10.000000,-7.362000,0.055500,0.084400,0.000000,0.063800,0.684000,106.446000,1,"With Rumours, Fleetwood Mac turned private tur...",46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8923,Minor Threat,Minor Threat,1987,['Rock'],"['Hardcore', 'Punk']",1,0,0,0,0,0,0,1,2290,532,4.59,160,34.99,0,NaN,0.294071,0.941286,3.642857,-6.782429,0.097100,0.028782,0.122042,0.312636,0.632857,148.792857,0,empty,36
8924,Carole King,Wrap Around Joy,1974,['Rock'],['Pop Rock'],0,0,0,0,0,0,0,1,2289,82,3.58,111,2.57,0,NaN,0.547833,0.452833,3.500000,-13.260250,0.029158,0.099133,0.000132,0.129717,0.650167,119.629833,0,empty,49
8925,Tom Jones,This Is Tom Jones,1969,['Pop'],"['Ballad', 'Vocal']",0,0,0,0,0,0,0,0,2292,31,3.50,84,2.33,0,NaN,0.354667,0.553917,4.083333,-10.294583,0.047683,0.457000,0.000111,0.212875,0.524667,133.280333,0,empty,54
8926,Pixies,Bossanova,1990,['Rock'],['Indie Rock'],0,0,0,0,0,0,0,0,2291,1286,4.26,185,34.50,0,NaN,0.472857,0.629857,5.000000,-12.273357,0.036579,0.045239,0.264102,0.143621,0.666429,132.973286,0,empty,33


In [ ]:
# clean artist data
#ValueError: 
#Joey Bada$$
#Expected end of text, found '$'  (at char 9), (line:1, col:10)

df_discogs_p4k_spotify['artist_x'] = df_discogs_p4k_spotify['artist_x'].str.replace(r"\$\$","SS", regex = True)
df_discogs_p4k_spotify['artist_x'] = df_discogs_p4k_spotify['artist_x'].str.replace(r"\([0-9]\)","", regex = True)

In [ ]:
# clean artist data to facilitate merging, some artists names contain

checking_numbers = df_discogs_p4k_spotify[df_discogs_p4k_spotify['artist_x'].str.contains('\([0-9]\)')]
checking_numbers.shape

In [19]:
# clean artist data to facilitate merging, some artists names contain

checking_numbers = df_train[df_train['artist'].str.contains('\([0-9]\)')]
checking_numbers.shape

(327, 32)

In [ ]:
# clean artist data to facilitate merging, some artists names contain

checking_dollar = df_discogs[df_discogs['artist'].str.contains('\$\$')]
print (checking_dollar.shape)

In [ ]:
# clean artist data to facilitate merging, some artists names contain

checking_dollar = df_discogs_p4k_spotify[df_discogs_p4k_spotify['artist_x'].str.contains('\$\$')]
print (checking_dollar.shape)

In [ ]:
# artist counts

count_artist = df_discogs['artist'].nunique()
print(count_artist, " unique artists found in data")

In [ ]:
# artist counts

count_artist = df_discogs_p4k_spotify['artist_x'].nunique()
print(count_artist, " unique artists found in data")

In [ ]:
# clean release title data
df_discogs['title'] = df_discogs['title'].str.replace(r"\$\$","SS", regex = True)

In [ ]:
checking_dollar_title = df_discogs[df_discogs['title'].str.contains('\$\$')]
print (checking_dollar_title.shape)

In [ ]:
# clean release title data
df_discogs_p4k_spotify['title_x'] = df_discogs_p4k_spotify['title_x'].str.replace(r"\$\$","SS", regex = True)

In [ ]:
checking_dollar_title = df_discogs_p4k_spotify[df_discogs_p4k_spotify['title_x'].str.contains('\$\$')]
print (checking_dollar_title.shape)

In [ ]:
# title counts

count_title = df_discogs['title'].nunique()
print(count_title, " unique titles found in data")

In [ ]:
# title counts

count_title = df_discogs_p4k_spotify['title_x'].nunique()
print(count_title, " unique titles found in data")

In [ ]:
# create anonymized title and artist columns

In [ ]:
# convert all categorical columns to category type
# df_discogs.influential_album.astype("category")
# df_discogs.issue_type.astype("category")
# df_discogs.limited.astype("category")
# df_discogs.picture_disc.astype("category")
# df_discogs.box_set.astype("category")
# df_discogs.channels.astype("category")
# df_discogs.gatefold.astype("category")
# df_discogs.colored.astype("category")

In [ ]:
df_discogs.info()

In [ ]:
df_discogs_p4k_spotify.info()

In [27]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8928 entries, 0 to 8927
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   artist                  8928 non-null   object 
 1   title                   8928 non-null   object 
 2   year                    8928 non-null   int64  
 3   genres                  8928 non-null   object 
 4   styles                  8928 non-null   object 
 5   issue_type              8928 non-null   int64  
 6   limited                 8928 non-null   int64  
 7   picture_disc            8928 non-null   int64  
 8   numbered                8928 non-null   int64  
 9   promo                   8928 non-null   int64  
 10  channels                8928 non-null   int64  
 11  gatefold                8928 non-null   int64  
 12  colored                 8928 non-null   int64  
 13  haves                   8928 non-null   int64  
 14  wants                   8928 non-null   

In [ ]:
# some 14 rows have missing price_median, remove them
df_discogs = df_discogs[df_discogs['price_median'].notna()]

In [ ]:
# some 10 rows have missing price_median, remove them
df_discogs_p4k_spotify = df_discogs_p4k_spotify[df_discogs_p4k_spotify['price_median'].notna()]

In [ ]:
df_discogs.info()

In [ ]:
df_discogs_p4k_spotify.info()

In [ ]:
df_discogs.skew()

In [28]:
df_train.skew()

/var/folders/2k/lqwk1p1108jctdxn9x58f5qm0000gn/T/ipykernel_31613/1035533876.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_train.skew()


year                      0.149455
issue_type                1.237832
limited                   3.704754
picture_disc             15.655236
numbered                 14.479178
promo                    33.367274
channels                  0.994316
gatefold                  1.896119
colored                   0.501223
haves                     3.930950
wants                     4.454015
avg_rating               -0.461810
num_ratings               4.824562
price_median             14.089788
influential_album         2.331579
score_p4k                -1.068100
danceability              0.037644
energy                   -0.426141
key                       0.045871
loudness                 -0.801261
speechiness               4.839135
acousticness              0.982581
instrumentalness          1.796366
liveness                  2.756265
valence                  -0.166448
tempo                    -0.457025
influential_album2020     2.339719
age                      -0.149455
dtype: float64

In [ ]:
df_discogs_p4k_spotify.skew()

In [ ]:
# check outliers in price
boxplot = df_discogs.boxplot(column=['log_price_median'],figsize=(10,5))
boxplot

In [ ]:
# check outliers in price
boxplot = df_discogs_p4k_spotify.boxplot(column=['price_median'],figsize=(10,5))
boxplot

In [ ]:
# remove outliers based on z-score, withe values 3 standard deviations away

df_discogs_p4k_spotify.price_median.quantile(0.01)

In [ ]:
df_discogs_train = df_discogs[(df_discogs.log_price_median < df_discogs.log_price_median.quantile(0.95)) & (df_discogs.log_price_median > df_discogs.log_price_median.quantile(0.05)) ]

In [ ]:
df_discogs_train

In [ ]:
df_discogs_p4k_spotify_train = df_discogs_p4k_spotify[(df_discogs_p4k_spotify.price_median < df_discogs_p4k_spotify.price_median.quantile(0.95)) & (df_discogs_p4k_spotify.price_median > df_discogs_p4k_spotify.price_median.quantile(0.05)) ]

In [ ]:
df_discogs_p4k_spotify_train

In [ ]:
# check outliers in price
boxplot = df_discogs_train.boxplot(column=['log_price_median'],figsize=(10,5))
boxplot

In [ ]:
# check outliers in price
boxplot = df_discogs_p4k_spotify_train.boxplot(column=['price_median'],figsize=(10,5))
boxplot

In [ ]:
# check influence of rolling stones albums on price
boxplot = df_discogs_train.boxplot(column=['log_price_median'],by="influential_album",figsize=(10,5))
boxplot

In [ ]:
# check influence of rolling stones albums on price
boxplot = df_discogs_p4k_spotify_train.boxplot(column=['price_median'],by="influential_album",figsize=(10,5))
boxplot

In [ ]:
# check influence of rolling stones albums on collectibility
boxplot = df_discogs_train.boxplot(column=['log_haves'],by="influential_album",figsize=(10,5))
boxplot

In [ ]:
# check influence of rolling stones albums on collectibility
boxplot = df_discogs_p4k_spotify_train.boxplot(column=['haves'],by="influential_album",figsize=(10,5))
boxplot

In [ ]:
# check influence of rolling stones albums on buyers
boxplot = df_discogs_train.boxplot(column=['log_wants'],by="influential_album",figsize=(10,5))
boxplot

In [ ]:
# check influence of rolling stones albums on buyers
boxplot = df_discogs_p4k_spotify_train.boxplot(column=['wants'],by="influential_album",figsize=(10,5))
boxplot

In [ ]:
# check influence of rolling stones albums on rating
boxplot = df_discogs_train.boxplot(column=['log_avg_rating'],by="influential_album",figsize=(10,5))
boxplot

In [ ]:
# check influence of rolling stones albums on rating
boxplot = df_discogs_p4k_spotify_train.boxplot(column=['avg_rating'],by="influential_album",figsize=(10,5))
boxplot

In [ ]:
df_discogs_train.info()

In [ ]:
df_discogs_p4k_spotify_train.info()

In [ ]:
pd.set_option('display.max_columns', None)
df_discogs.describe()

In [ ]:
df_discogs_p4k_spotify_train.describe()

In [ ]:
features_spotify = ["issue_type", "limited", "picture_disc","numbered","promo","channels","gatefold","colored","haves","wants","avg_rating","num_ratings","influential_album","age","danceability","energy","key","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo"]

df_copy_spotify = df_discogs_p4k_spotify_train
df_copy_spotify.dropna()
df_copy_spotify

In [ ]:
# artist counts

count_genres = df_copy_spotify['genres'].nunique()
print(count_genres, " unique genres found in data")
df_copy_spotify['genres_list'] = df_copy_spotify.genres.apply(lambda x: x.replace(' ', '').replace("'", '')[1:-1].split(','))
df_copy_spotify.genres_list

In [32]:
# artist counts

count_genres = df_train['genres'].nunique()
print(count_genres, " unique genres found in data")
df_train['genres_list'] = df_train.genres.apply(lambda x: x.replace(' ', '').replace("'", '')[1:-1].split(','))
df_train.genres_list

233  unique genres found in data


/var/folders/2k/lqwk1p1108jctdxn9x58f5qm0000gn/T/ipykernel_31613/116321348.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['genres_list'] = df_train.genres.apply(lambda x: x.replace(' ', '').replace("'", '')[1:-1].split(','))


0       [Electronic, Funk/Soul, Pop]
1                           [HipHop]
2                   [Funk/Soul, Pop]
3                             [Rock]
4                             [Rock]
                    ...             
8923                          [Rock]
8924                          [Rock]
8925                           [Pop]
8926                          [Rock]
8927                  [Stage&Screen]
Name: genres_list, Length: 8928, dtype: object

In [33]:
df_train["genres_list"].explode().unique()

array(['Electronic', 'Funk/Soul', 'Pop', 'HipHop', 'Rock', 'Stage&Screen',
       'Folk', 'World', '&Country', 'Jazz', 'Classical', 'Blues',
       'Non-Music', 'Latin', 'Reggae', '"Childrens"', 'Brass&Military'],
      dtype=object)

In [34]:
df_train

,artist,title,year,genres,styles,issue_type,limited,picture_disc,numbered,promo,channels,gatefold,colored,haves,wants,avg_rating,num_ratings,price_median,influential_album,score_p4k,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,influential_album2020,influential_album_text,age,genres_list
0,Daft Punk,Random Access Memories,2013,"['Electronic', 'Funk / Soul', 'Pop']","['Disco', 'Funk', 'Synth-pop']",0,0,0,0,0,1,0,0,53521,18838,4.56,6337,45.00,0,8.8,0.686462,0.585846,7.538462,-10.704077,0.044315,0.287906,0.373620,0.132877,0.488231,114.471077,1,Having played a massive role in the rise of ED...,10,"[Electronic, Funk/Soul, Pop]"
1,Kendrick Lamar,"Good Kid, M.A.A.d City",2012,['Hip Hop'],['Conscious'],0,0,0,0,0,0,0,0,41176,12493,4.66,2957,20.90,0,9.5,0.601294,0.676706,4.529412,-7.811941,0.268318,0.123219,0.000669,0.311412,0.398206,116.045059,0,empty,11,[HipHop]
2,Michael Jackson,Thriller,1982,"['Funk / Soul', 'Pop']","['Contemporary R&B', 'Disco', 'Soul']",0,0,0,0,0,1,1,1,40452,9254,4.48,2744,15.06,1,NaN,0.727100,0.585233,6.266667,-9.707967,0.349890,0.375693,0.044753,0.296793,0.608980,110.307933,1,Michael Jackson towered over the Eighties the ...,41,"[Funk/Soul, Pop]"
3,Pink Floyd,The Dark Side Of The Moon,2016,['Rock'],"['Prog Rock', 'Psychedelic Rock', 'Classic Rock']",1,0,0,0,0,1,1,0,38314,3961,4.83,4201,23.62,0,NaN,0.383000,0.400450,6.000000,-15.701900,0.058660,0.390310,0.595929,0.237680,0.272830,120.066200,0,empty,7,[Rock]
4,Fleetwood Mac,Rumours,1977,['Rock'],"['Soft Rock', 'Classic Rock']",0,0,0,0,0,0,0,1,35108,8351,4.50,2713,21.84,1,10.0,0.857000,0.812000,10.000000,-7.362000,0.055500,0.084400,0.000000,0.063800,0.684000,106.446000,1,"With Rumours, Fleetwood Mac turned private tur...",46,[Rock]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8923,Minor Threat,Minor Threat,1987,['Rock'],"['Hardcore', 'Punk']",1,0,0,0,0,0,0,1,2290,532,4.59,160,34.99,0,NaN,0.294071,0.941286,3.642857,-6.782429,0.097100,0.028782,0.122042,0.312636,0.632857,148.792857,0,empty,36,[Rock]
8924,Carole King,Wrap Around Joy,1974,['Rock'],['Pop Rock'],0,0,0,0,0,0,0,1,2289,82,3.58,111,2.57,0,NaN,0.547833,0.452833,3.500000,-13.260250,0.029158,0.099133,0.000132,0.129717,0.650167,119.629833,0,empty,49,[Rock]
8925,Tom Jones,This Is Tom Jones,1969,['Pop'],"['Ballad', 'Vocal']",0,0,0,0,0,0,0,0,2292,31,3.50,84,2.33,0,NaN,0.354667,0.553917,4.083333,-10.294583,0.047683,0.457000,0.000111,0.212875,0.524667,133.280333,0,empty,54,[Pop]
8926,Pixies,Bossanova,1990,['Rock'],['Indie Rock'],0,0,0,0,0,0,0,0,2291,1286,4.26,185,34.50,0,NaN,0.472857,0.629857,5.000000,-12.273357,0.036579,0.045239,0.264102,0.143621,0.666429,132.973286,0,empty,33,[Rock]


In [ ]:
# artist counts

count_styles = df_copy_spotify['styles'].nunique()
print(count_styles, " unique styles found in data")
df_copy_spotify['styles_list'] = df_copy_spotify.styles.apply(lambda x: x.replace(' ', '').replace("'", '')[1:-1].split(','))
df_copy_spotify.styles_list

In [35]:
# artist counts

count_styles = df_train['styles'].nunique()
print(count_styles, " unique styles found in data")
df_train['styles_list'] = df_train.styles.apply(lambda x: x.replace(' ', '').replace("'", '')[1:-1].split(','))
df_train.styles_list

2813  unique styles found in data


/var/folders/2k/lqwk1p1108jctdxn9x58f5qm0000gn/T/ipykernel_31613/188857641.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['styles_list'] = df_train.styles.apply(lambda x: x.replace(' ', '').replace("'", '')[1:-1].split(','))


0                       [Disco, Funk, Synth-pop]
1                                    [Conscious]
2                 [ContemporaryR&B, Disco, Soul]
3       [ProgRock, PsychedelicRock, ClassicRock]
4                        [SoftRock, ClassicRock]
                          ...                   
8923                            [Hardcore, Punk]
8924                                   [PopRock]
8925                             [Ballad, Vocal]
8926                                 [IndieRock]
8927                       [Musical, Soundtrack]
Name: styles_list, Length: 8928, dtype: object

In [36]:
df_train["styles_list"].explode().unique()

array(['Disco', 'Funk', 'Synth-pop', 'Conscious', 'ContemporaryR&B',
       'Soul', 'ProgRock', 'PsychedelicRock', 'ClassicRock', 'SoftRock',
       'AlternativeRock', 'BluesRock', 'CountryRock', 'MinneapolisSound',
       'PopRock', 'Soundtrack', 'Grunge', 'FolkRock', 'Glam', 'ArtRock',
       'IndieRock', 'NewWave', 'PowerPop', 'Folk', 'Britpop', 'Acoustic',
       'HeavyMetal', 'Vocal', 'HardRock', 'House', 'Country', 'Rock&Roll',
       'TripHop', 'Downtempo', 'Experimental', 'HardcoreHip-Hop',
       'BoomBap', 'JazzyHip-Hop', 'Leftfield', 'Lo-Fi', 'Score',
       'Neo-Romantic', '', 'Ballad', 'ArenaRock', 'Modal', 'CoolJazz',
       'GarageRock', 'JanglePop', 'IndiePop', 'Post-Punk', 'PostRock',
       'PopRap', 'Jazz-Rock', 'Bluegrass', 'Punk', 'Thrash', 'SpeedMetal',
       'G-Funk', 'Soul-Jazz', 'PianoBlues', 'NeoSoul', 'Electro',
       'Ambient', 'IDM', 'Surf', 'Dialogue', 'DreamPop', 'Trap', 'Fusion',
       'Jazz-Funk', 'FunkMetal', 'Techno', 'ContemporaryJazz',
       'Ps

In [46]:
df_train = df_train.drop(columns=["influential_album"], axis = 1)

In [47]:
df_train

,artist,title,issue_type,limited,picture_disc,numbered,promo,channels,gatefold,colored,haves,wants,avg_rating,num_ratings,price_median,score_p4k,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,influential_album2020,influential_album_text,age,genres_list,styles_list
0,Daft Punk,Random Access Memories,0,0,0,0,0,1,0,0,53521,18838,4.56,6337,45.00,8.8,0.686462,0.585846,7.538462,-10.704077,0.044315,0.287906,0.373620,0.132877,0.488231,114.471077,1,Having played a massive role in the rise of ED...,10,"[Electronic, Funk/Soul, Pop]","[Disco, Funk, Synth-pop]"
1,Kendrick Lamar,"Good Kid, M.A.A.d City",0,0,0,0,0,0,0,0,41176,12493,4.66,2957,20.90,9.5,0.601294,0.676706,4.529412,-7.811941,0.268318,0.123219,0.000669,0.311412,0.398206,116.045059,0,empty,11,[HipHop],[Conscious]
2,Michael Jackson,Thriller,0,0,0,0,0,1,1,1,40452,9254,4.48,2744,15.06,NaN,0.727100,0.585233,6.266667,-9.707967,0.349890,0.375693,0.044753,0.296793,0.608980,110.307933,1,Michael Jackson towered over the Eighties the ...,41,"[Funk/Soul, Pop]","[ContemporaryR&B, Disco, Soul]"
3,Pink Floyd,The Dark Side Of The Moon,1,0,0,0,0,1,1,0,38314,3961,4.83,4201,23.62,NaN,0.383000,0.400450,6.000000,-15.701900,0.058660,0.390310,0.595929,0.237680,0.272830,120.066200,0,empty,7,[Rock],"[ProgRock, PsychedelicRock, ClassicRock]"
4,Fleetwood Mac,Rumours,0,0,0,0,0,0,0,1,35108,8351,4.50,2713,21.84,10.0,0.857000,0.812000,10.000000,-7.362000,0.055500,0.084400,0.000000,0.063800,0.684000,106.446000,1,"With Rumours, Fleetwood Mac turned private tur...",46,[Rock],"[SoftRock, ClassicRock]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8923,Minor Threat,Minor Threat,1,0,0,0,0,0,0,1,2290,532,4.59,160,34.99,NaN,0.294071,0.941286,3.642857,-6.782429,0.097100,0.028782,0.122042,0.312636,0.632857,148.792857,0,empty,36,[Rock],"[Hardcore, Punk]"
8924,Carole King,Wrap Around Joy,0,0,0,0,0,0,0,1,2289,82,3.58,111,2.57,NaN,0.547833,0.452833,3.500000,-13.260250,0.029158,0.099133,0.000132,0.129717,0.650167,119.629833,0,empty,49,[Rock],[PopRock]
8925,Tom Jones,This Is Tom Jones,0,0,0,0,0,0,0,0,2292,31,3.50,84,2.33,NaN,0.354667,0.553917,4.083333,-10.294583,0.047683,0.457000,0.000111,0.212875,0.524667,133.280333,0,empty,54,[Pop],"[Ballad, Vocal]"
8926,Pixies,Bossanova,0,0,0,0,0,0,0,0,2291,1286,4.26,185,34.50,NaN,0.472857,0.629857,5.000000,-12.273357,0.036579,0.045239,0.264102,0.143621,0.666429,132.973286,0,empty,33,[Rock],[IndieRock]


In [48]:
df_train.to_csv("datasets/training.csv")

In [ ]:
df_copy_spotify.isnull().sum()

In [ ]:
mlb = MultiLabelBinarizer()

df = pd.DataFrame(mlb.fit_transform(df_copy_spotify['genres_list']), index=df_copy_spotify.index)
df

In [ ]:
features_spotify = ["issue_type", "limited", "picture_disc","numbered","promo","channels","gatefold","colored","haves","wants","avg_rating","num_ratings","influential_album","age","danceability","energy","key","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo"]
targets = ["price_median","haves","wants"]

target = targets[-1]
print(target)
print(targets)
features = [feature for feature in features_spotify if feature != target]
print(features)
targets.pop()
print(targets)

In [ ]:
!python -m spacy download en_core_web_md 

In [ ]:
#TF-IDF

import spacy
nlp = spacy.load('en_core_web_md')

import numpy as np
from spacy.lang.en.stop_words import STOP_WORDS
import re

def clean_tfidf(string_list):
    clean_string_list = []
    for string in string_list:
        string = string.lower()
        string = re.sub(r'[^a-z]', ' ',string)
        tokens = nlp(string)
    
        sent_clean = []

        for token in tokens:
            if token.is_stop == False and token.text.isspace() == False:
                sent_clean.append(token.lemma_)
                
        clean_string_list.append(' '.join(sent_clean))
        
    return clean_string_list 

text_train = clean_tfidf(df_discogs_p4k_spotify["influential_album_text"])


In [ ]:
text_train[:3]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import Adam

def experiment(df,name_of_experiment,feature_set="discogs",genres_styles=False,target_predict="price_median"):
    rows = []
    experiment = dict()
    
    experiment["name"] = name_of_experiment
    experiment["shape"] = df.shape
    
    print("running experiment ", name_of_experiment, " with feature set : ", feature_set)
    print("training dataset shape : ", df.shape)
    
    if target_predict == "price_median":
    
        if genres_styles:
            features = ["issue_type", "limited", "picture_disc","numbered","promo","channels","gatefold","colored","haves","wants","avg_rating","num_ratings","influential_album","age","genres_list","styles_list"]
            features_spotify = ["issue_type", "limited", "picture_disc","numbered","promo","channels","gatefold","colored","haves","wants","avg_rating","num_ratings","influential_album","age","danceability","energy","key","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo","genres_list","styles_list"]
            features_spotify_p4k = ["issue_type", "limited", "picture_disc","numbered","promo","channels","gatefold","colored","haves","wants","avg_rating","num_ratings","influential_album","age","score","danceability","energy","key","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo","genres_list","styles_list"]
        else:
            features = ["issue_type", "limited", "picture_disc","numbered","promo","channels","gatefold","colored","haves","wants","avg_rating","num_ratings","influential_album","age"]
            features_spotify = ["issue_type", "limited", "picture_disc","numbered","promo","channels","gatefold","colored","haves","wants","avg_rating","num_ratings","influential_album","age","danceability","energy","key","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo"]
            features_spotify_p4k = ["issue_type", "limited", "picture_disc","numbered","promo","channels","gatefold","colored","haves","wants","avg_rating","num_ratings","influential_album","age","score","danceability","energy","key","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo"]
        target = ["price_median"]
    elif target_predict == "haves":
        if genres_styles:
            features = ["issue_type", "limited", "picture_disc","numbered","promo","channels","gatefold","colored","wants","avg_rating","num_ratings","influential_album","age","genres_list","styles_list"]
            features_spotify = ["issue_type", "limited", "picture_disc","numbered","promo","channels","gatefold","colored","wants","avg_rating","num_ratings","influential_album","age","danceability","energy","key","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo","genres_list","styles_list"]
            features_spotify_p4k = ["issue_type", "limited", "picture_disc","numbered","promo","channels","gatefold","colored","wants","avg_rating","num_ratings","influential_album","age","score","danceability","energy","key","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo","genres_list","styles_list"]
        else:
            features = ["issue_type", "limited", "picture_disc","numbered","promo","channels","gatefold","colored","wants","avg_rating","num_ratings","influential_album","age"]
            features_spotify = ["issue_type", "limited", "picture_disc","numbered","promo","channels","gatefold","colored","wants","avg_rating","num_ratings","influential_album","age","danceability","energy","key","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo"]
            features_spotify_p4k = ["issue_type", "limited", "picture_disc","numbered","promo","channels","gatefold","colored","wants","avg_rating","num_ratings","influential_album","age","score","danceability","energy","key","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo"]
        target = ["haves"]       
        
    
    if feature_set == "spotify":
        df = df[features_spotify + target].dropna()
        X = df[features_spotify]
        y = df[target]
        print("training dataset shape after dropping na : ", df.shape)
    else:
        #select features and target from dataframe
        print("feature set is discogs")
        X = df[features]
        y = df[target]
    
    if genres_styles:
        mlb_genres = MultiLabelBinarizer()
        mlb_genres.fit(X['genres_list'])
        
        mlb_styles = MultiLabelBinarizer()
        mlb_styles.fit(X['styles_list'])

    print("Input data columns:", X.columns)
    #split as training and test
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=7)
    
    #standardize numeric data using log-transform
    X_train["age"] = np.log(X_train["age"])
    if target_predict != "haves":
        X_train["haves"] = np.log(X_train["haves"])
    X_train["wants"] = np.log(X_train["wants"])
    X_train["avg_rating"] = np.log(X_train["avg_rating"])
    X_train["num_ratings"] = np.log(X_train["num_ratings"])
    
    y_train = np.log(y_train)
    
    X_test["age"] = np.log(X_test["age"])
    if target_predict != "haves":
        X_test["haves"] = np.log(X_test["haves"])
    X_test["wants"] = np.log(X_test["wants"])
    X_test["avg_rating"] = np.log(X_test["avg_rating"])
    X_test["num_ratings"] = np.log(X_test["num_ratings"])
    
    y_test = np.log(y_test)
    
    #when testing genres and styles, do multilabel binarization
    if genres_styles:
        
        genres_train_var = pd.DataFrame(mlb_genres.fit_transform(X_train['genres_list']), index=X_train.index)
        print(genres_train_var.shape, type(genres_train_var))
        X_train = pd.concat([X_train, genres_train_var], axis=1)
        print("X_train after concat : ", X_train.shape)
        
        styles_train_var = pd.DataFrame(mlb_styles.fit_transform(X_train['styles_list']), index=X_train.index)
        print(styles_train_var.shape)
        X_train = pd.concat([X_train, styles_train_var], axis=1)
        print("X_train after concat : ", X_train.shape)
        print(X_train.columns[:20])

        genres_test_var = pd.DataFrame(mlb_genres.transform(X_test['genres_list']), index=X_test.index)
        print(genres_test_var.shape)
        X_test = pd.concat([X_test, genres_test_var], axis=1)
        print("X_test after concat : ", X_test.shape)

        styles_test_var = pd.DataFrame(mlb_styles.transform(X_test['styles_list']), index=X_test.index)
        print(styles_test_var.shape)
        X_test = pd.concat([X_test, styles_test_var], axis=1)
        print("X_test after concat : ", X_test.shape)
        
        X_train = X_train.drop(['genres_list', 'styles_list'], axis = 1)
        X_test = X_test.drop(['genres_list', 'styles_list'], axis = 1)
    units = 28
    input_dim = 14
    
    if feature_set == "spotify":
        units = 48
        input_dim = 24
        #standardise loudness, it has negative values
        scale_train = StandardScaler().fit(X_train[["loudness"]])
        # transform the training data column
        X_train["loudness"] = scale_train.transform(X_train[["loudness"]])
    
        # transform the testing data column
        X_test["loudness"] = scale_train.transform(X_test[["loudness"]])
    #one-hot encoding of categoricals
    #X_train = pd.get_dummies(X_train)
    #X_test = pd.get_dummies(X_test)
    print(X_train.shape)
    print(X_test.shape)
    print(X_train.dtypes)
    
    
    models = dict()
    results = dict()
    
    models["LR"] = (LinearRegression, {"fit_intercept" : False})
    models["KN3"] = (KNeighborsRegressor, {"n_neighbors" : 3, "weights" : "distance"})
    models["KN5"] = (KNeighborsRegressor, {"n_neighbors" : 5, "weights" : "distance"})
    models["KN7"] = (KNeighborsRegressor, {"n_neighbors" : 7, "weights" : "distance"})
    models["RF14"] = (RandomForestRegressor, {"n_estimators" : 14})
    models["RF28"] = (RandomForestRegressor, {"n_estimators" : 28})
    models["RF50"] = (RandomForestRegressor, {"n_estimators" : 50})
    models["RF100"] = (RandomForestRegressor, {"n_estimators" : 100})
    models["RF150"] = (RandomForestRegressor, {"n_estimators" : 150})
    models["RF250"] = (RandomForestRegressor, {"n_estimators" : 250})
    models["NN"] = (MLPRegressor, {"solver":"lbfgs","activation":"relu"})
    print(models)
    
    started = time()
    print("Starting training. Time : ", started)
    
    def get_scores(y_test, y_pred):
        est_test = np.log1p(np.maximum(0, y_test))
    
        est_pred = np.log1p(np.maximum(0, y_pred))
        return (  
        # R2  
        round(r2_score(est_test, est_pred), 3),  
        # RMSE  
        round(sqrt(mean_squared_error(est_test, est_pred)), 3),  
        # MAPE  
        round(mean_absolute_percentage_error(est_test, est_pred), 3)
        )
    
    
    
    model = Sequential()
    model.add(Dense(X_train.shape[1]*2, input_dim=X_train.shape[1], activation='tanh'))
    model.add(Dense(X_train.shape[1]*2, activation='tanh'))
    model.add(Dense(1, activation='linear'))
    
    optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    
    model.fit(X_train, y_train, epochs=20, batch_size=1, verbose=2)
    
    y_pred = model.predict(X_test)
    
    r2, rmse, mape = get_scores(y_test,y_pred)
    rows.append(["NN_Sequential","vinyl + mood","Adam, 16+16, epochs = 20",r2, rmse, mape])
    
    
    
    for name, model in models.items():
        start = time()
        print("Training ", name)
        tested_model = model[0](**model[1])
        tested_model.fit(X_train, y_train)
        
        y_pred = tested_model.predict(X_test).tolist()
        
        r2, rmse, mape = get_scores(y_test, y_pred)  
        results[f"{name}_r2"] = r2  
        results[f"{name}_rmse"] = rmse  
        results[f"{name}_mape"] = mape
        
        rows.append([name,"vinyl + mood",str(model),r2, rmse, mape])
        
        print("Training finished. Elapsed : ", str(round(time()-start,2)))
    
    print("Model training finished. It took total ", str(round(time()-start,2)))
    
    results = pd.DataFrame(rows, columns=["Model", "Feature Set","Params","R2","RMSE","MAPE"])
    return results
    

In [ ]:
results = experiment(df_copy_spotify, "spotify+discogs", "spotify", True)

In [ ]:
results

In [ ]:
results_haves = experiment(df_copy_spotify, "spotify+discogs", "spotify", True, target_predict = "haves")

In [ ]:
results_haves

In [ ]:
results_only_vinyl = experiment(df_copy_spotify, "discogs", "discogs")

In [ ]:
results_only_vinyl